In [15]:
from utils import display_desktop
display_desktop()

HTML(value='<a href="https://jupyter.intel4coro.de//user/sourinkrpal-my-franka-lab-s0cunn9y/desktop"  class="j…

In [31]:
import os
import time
import threading
import mujoco
import mujoco.viewer
import numpy as np
from ipywidgets import Button, HBox, Output
# -------------------------
# Load robot
# -------------------------
robot = "franka_emika_panda"
XML_PATH = f"{os.getcwd()}/mujoco_menagerie/{robot}/scene.xml"
m = mujoco.MjModel.from_xml_path(XML_PATH)
d = mujoco.MjData(m)
# -------------------------
# Shared control flag
# -------------------------
running = False
viewer_thread = None
# -------------------------
# Simulation function
# -------------------------
def run_sim(m, d):
    global running
    with mujoco.viewer.launch_passive(
        m, d,
        show_left_ui=False,
        show_right_ui=False,
    ) as viewer:
        # ----------- Remove overlays / visuals -----------
        with viewer.lock():
            # disable shadows & reflections
            viewer.user_scn.flags[mujoco.mjtRndFlag.mjRND_SHADOW] = 0
            viewer.user_scn.flags[mujoco.mjtRndFlag.mjRND_REFLECTION] = 0
            # disable contact points (overlay)
            viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = 0
        viewer.sync()
        start = time.time()
        while running and viewer.is_running():
            step_start = time.time()
            # Random control
            ctrl_range = m.actuator_ctrlrange
            d.ctrl = np.random.uniform(ctrl_range[:,0], ctrl_range[:,1])
            mujoco.mj_step(m, d)
            viewer.sync()
            # Real-time pacing (keep for stability)
            dt = m.opt.timestep - (time.time() - step_start)
            if dt > 0:
                time.sleep(dt)
    running = False
# -------------------------
# Button handlers
# -------------------------
def start_clicked(b):
    global running, viewer_thread
    if running:
        out.append_stdout("Simulation already running.\n")
        return
    out.append_stdout("Starting simulation...\n")
    running = True
    viewer_thread = threading.Thread(target=run_sim, args=(m, d), daemon=True)
    viewer_thread.start()
def stop_clicked(b):
    global running
    if running:
        out.append_stdout("Stopping simulation...\n")
        running = False
    else:
        out.append_stdout("Simulation is not running.\n")
# -------------------------
# UI Buttons
# -------------------------
start_button = Button(description="Start Simulation", button_style="success")
stop_button = Button(description="Stop Simulation", button_style="danger")
start_button.on_click(start_clicked)
stop_button.on_click(stop_clicked)
out = Output()
display(HBox([start_button, stop_button]))
display(out)

Output()